# 📦 2.2 Pydantic 資料驗證

Pydantic 是 FastAPI 的資料驗證引擎，讓我們深入學習！

---

## 🎯 學習目標

- ✅ 使用 `BaseModel` 定義資料模型
- ✅ 掌握欄位驗證器 `Field`
- ✅ 自訂驗證邏輯 `@field_validator`
- ✅ 模型繼承與巢狀結構
- ✅ 實際應用於 FastAPI

---

## 🤔 為什麼需要 Pydantic？

處理 API 請求時，我們面對的是**外部世界的資料**——它可能是錯的、缺的、格式不對的。

### 沒有驗證的危險

```python
# 前端傳來的 JSON
{"name": "", "age": -5, "email": "not-an-email"}

# 如果你不驗證就存進資料庫...
# 💥 資料品質崩壞
# 💥 後續程式碼會出錯
# 💥 使用者體驗很差
```

### Pydantic 幫你做的四件事

| 工作 | 沒有 Pydantic | 有 Pydantic |
|------|--------------|-------------|
| **驗證格式** | 自己寫 if 檢查 | 自動檢查 |
| **驗證類型** | `isinstance()` 到處寫 | 宣告即驗證 |
| **驗證內容** | 業務邏輯散落各處 | `@field_validator` 集中管理 |
| **轉換資料** | 手動 `json.loads()` | 自動處理 |

### 🎯 核心概念

> **Pydantic 的哲學**：用 Python 類別「描述」你期望的資料長什麼樣子，剩下的交給它處理。

In [1]:
# 沒有 Pydantic：手動驗證超麻煩

def create_user_manual(data: dict):
    # 檢查必填欄位
    if "name" not in data:
        raise ValueError("缺少 name 欄位")
    if "email" not in data:
        raise ValueError("缺少 email 欄位")
    
    # 檢查類型
    if not isinstance(data["name"], str):
        raise ValueError("name 必須是字串")
    if not isinstance(data["email"], str):
        raise ValueError("email 必須是字串")
    
    # 檢查格式
    if "@" not in data["email"]:
        raise ValueError("email 格式不正確")
    
    # 設定預設值
    age = data.get("age", 0)
    
    return {"name": data["name"], "email": data["email"], "age": age}

print("手動驗證需要寫很多程式碼... 😫")

手動驗證需要寫很多程式碼... 😫


In [2]:
# 使用 Pydantic：簡潔又強大

from pydantic import BaseModel, EmailStr
from typing import Optional

class User(BaseModel):
    name: str
    email: EmailStr
    age: Optional[int] = 0

# 自動驗證！
user = User(name="小明", email="ming@example.com", age=18)
print(f"✅ 建立成功: {user}")

# 從字典建立
data = {"name": "小華", "email": "hua@example.com"}
user2 = User(**data)
print(f"✅ 從字典建立: {user2}")

✅ 建立成功: name='小明' email='ming@example.com' age=18
✅ 從字典建立: name='小華' email='hua@example.com' age=0


---

## 📝 BaseModel 基本用法

所有 Pydantic 模型都繼承自 `BaseModel`。

In [3]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

class Product(BaseModel):
    """商品模型"""
    id: int
    name: str
    price: float
    description: Optional[str] = None
    in_stock: bool = True
    created_at: datetime = datetime.now()

# 建立商品
product = Product(
    id=1,
    name="Python 書籍",
    price=599.0,
    description="FastAPI 入門教學"
)

print(f"商品: {product}")
print(f"\n各欄位:")
print(f"  ID: {product.id}")
print(f"  名稱: {product.name}")
print(f"  價格: {product.price}")
print(f"  庫存: {product.in_stock}")

商品: id=1 name='Python 書籍' price=599.0 description='FastAPI 入門教學' in_stock=True created_at=datetime.datetime(2025, 12, 30, 17, 22, 19, 187531)

各欄位:
  ID: 1
  名稱: Python 書籍
  價格: 599.0
  庫存: True


In [4]:
# 模型轉換方法

print("📦 模型轉換方法：\n")

# 轉為字典
product_dict = product.model_dump()
print(f"model_dump(): {product_dict}")

# 轉為 JSON 字串
product_json = product.model_dump_json()
print(f"\nmodel_dump_json(): {product_json}")

# 從 JSON 字串建立
json_str = '{"id": 2, "name": "筆記本", "price": 50.0}'
product2 = Product.model_validate_json(json_str)
print(f"\nmodel_validate_json(): {product2}")

📦 模型轉換方法：

model_dump(): {'id': 1, 'name': 'Python 書籍', 'price': 599.0, 'description': 'FastAPI 入門教學', 'in_stock': True, 'created_at': datetime.datetime(2025, 12, 30, 17, 22, 19, 187531)}

model_dump_json(): {"id":1,"name":"Python 書籍","price":599.0,"description":"FastAPI 入門教學","in_stock":true,"created_at":"2025-12-30T17:22:19.187531"}

model_validate_json(): id=2 name='筆記本' price=50.0 description=None in_stock=True created_at=datetime.datetime(2025, 12, 30, 17, 22, 19, 187531)


---

## 🔒 Field - 欄位驗證

`BaseModel` 只能驗證類型，但實際需求更複雜：

- 名字不能超過 50 字
- 年齡不能是負數
- Email 要符合特定格式
- 分數要在 0-100 之間

`Field` 就是用來設定這些**額外驗證規則**的。

### Field 的運作方式

```python
name: str = Field(min_length=2, max_length=50)
#     ↑      ↑              ↑
#   類型   Field()    驗證規則（可多個）
```

### 常見驗證需求對照表

| 需求 | Field 參數 | 範例 |
|------|-----------|------|
| 字串最短長度 | `min_length` | `Field(min_length=2)` |
| 字串最長長度 | `max_length` | `Field(max_length=100)` |
| 數值 >= N | `ge` (greater or equal) | `Field(ge=0)` |
| 數值 > N | `gt` (greater than) | `Field(gt=0)` |
| 數值 <= N | `le` (less or equal) | `Field(le=100)` |
| 數值 < N | `lt` (less than) | `Field(lt=100)` |
| 符合正則表達式 | `pattern` | `Field(pattern=r'^\d+$')` |
| 設定預設值 | `default` | `Field(default=0)` |

In [5]:
from pydantic import BaseModel, Field
from typing import Optional

class User(BaseModel):
    """用戶模型 - 使用 Field 驗證"""
    
    name: str = Field(
        min_length=2,
        max_length=50,
        description="用戶名稱，2-50 字元"
    )
    
    age: int = Field(
        ge=0,           # greater than or equal (>=)
        le=150,         # less than or equal (<=)
        description="年齡，0-150"
    )
    
    email: str = Field(
        pattern=r'^[\w.-]+@[\w.-]+\.\w+$',  # 正則表達式
        description="Email 格式"
    )
    
    score: float = Field(
        default=0.0,
        ge=0,
        le=100,
        description="分數 0-100"
    )

# 正確的資料
user = User(name="小明", age=18, email="ming@example.com")
print(f"✅ 正確: {user}")

✅ 正確: name='小明' age=18 email='ming@example.com' score=0.0


In [6]:
from pydantic import ValidationError

# 測試驗證錯誤
print("❌ 驗證錯誤測試：\n")

test_cases = [
    {"name": "A", "age": 18, "email": "test@test.com"},  # name 太短
    {"name": "小明", "age": -5, "email": "test@test.com"},  # age 負數
    {"name": "小明", "age": 18, "email": "invalid-email"},  # email 格式錯誤
]

for i, data in enumerate(test_cases, 1):
    try:
        User(**data)
    except ValidationError as e:
        error = e.errors()[0]
        print(f"測試 {i}: {error['loc'][0]} - {error['msg']}")

❌ 驗證錯誤測試：

測試 1: name - String should have at least 2 characters
測試 2: age - Input should be greater than or equal to 0
測試 3: email - String should match pattern '^[\w.-]+@[\w.-]+\.\w+$'


### Field 常用參數

| 參數 | 說明 | 範例 |
|------|------|------|
| `default` | 預設值 | `Field(default=0)` |
| `min_length` | 字串最小長度 | `Field(min_length=2)` |
| `max_length` | 字串最大長度 | `Field(max_length=50)` |
| `ge` | 大於等於 | `Field(ge=0)` |
| `gt` | 大於 | `Field(gt=0)` |
| `le` | 小於等於 | `Field(le=100)` |
| `lt` | 小於 | `Field(lt=100)` |
| `pattern` | 正則表達式 | `Field(pattern=r'^\d+$')` |
| `description` | 欄位說明 | `Field(description="...")` |
| `examples` | 範例值 | `Field(examples=["example"])` |

---

## ✨ 自訂驗證器

`Field` 的參數不夠用？用 `@field_validator` 寫自己的驗證邏輯！

### 什麼時候需要自訂驗證器？

| 情境 | 範例 |
|------|------|
| **格式檢查** | 密碼要有大寫、小寫、數字 |
| **業務邏輯** | 結束日期必須晚於開始日期 |
| **資料清理** | 去除前後空白、轉小寫 |
| **複雜規則** | 根據國家驗證電話格式 |

### 驗證器的基本結構

```python
@field_validator('欄位名稱')
@classmethod
def 驗證函式名稱(cls, v: 欄位類型) -> 欄位類型:
    if 不符合條件:
        raise ValueError('錯誤訊息')
    return v  # 可以回傳處理後的值
```

### 重要觀念

1. **必須加 `@classmethod`** - 這是 Pydantic v2 的要求
2. **參數 `v` 是欄位值** - 驗證邏輯針對這個值
3. **可以修改並回傳** - 例如 `return v.strip()` 去除空白
4. **用 `ValueError` 報錯** - Pydantic 會捕獲並轉成驗證錯誤

In [7]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    name: str
    email: str
    password: str
    
    @field_validator('name')
    @classmethod
    def name_must_not_be_empty(cls, v: str) -> str:
        """名稱不能是空白"""
        if not v.strip():
            raise ValueError('名稱不能是空白')
        return v.strip()  # 回傳處理後的值
    
    @field_validator('email')
    @classmethod
    def email_must_be_valid(cls, v: str) -> str:
        """Email 必須包含 @"""
        if '@' not in v:
            raise ValueError('Email 格式不正確')
        return v.lower()  # 轉小寫
    
    @field_validator('password')
    @classmethod
    def password_strength(cls, v: str) -> str:
        """密碼強度檢查"""
        if len(v) < 8:
            raise ValueError('密碼至少需要 8 個字元')
        if not any(c.isupper() for c in v):
            raise ValueError('密碼需要至少一個大寫字母')
        if not any(c.isdigit() for c in v):
            raise ValueError('密碼需要至少一個數字')
        return v

# 測試
try:
    user = User(
        name="  小明  ",
        email="MING@Example.com",
        password="MyPassword123"
    )
    print(f"✅ 建立成功:")
    print(f"   名稱: '{user.name}'")
    print(f"   Email: '{user.email}'")
except ValidationError as e:
    print(f"❌ 錯誤: {e}")

✅ 建立成功:
   名稱: '小明'
   Email: 'ming@example.com'


In [8]:
# 測試密碼驗證失敗
test_passwords = [
    "short",        # 太短
    "alllowercase1", # 沒有大寫
    "NoNumbers",    # 沒有數字
    "Valid123Pass"  # 正確
]

print("🔐 密碼驗證測試：\n")

for pwd in test_passwords:
    try:
        user = User(name="Test", email="test@test.com", password=pwd)
        print(f"✅ '{pwd}' - 通過")
    except ValidationError as e:
        error_msg = e.errors()[0]['msg']
        print(f"❌ '{pwd}' - {error_msg}")

🔐 密碼驗證測試：

❌ 'short' - Value error, 密碼至少需要 8 個字元
❌ 'alllowercase1' - Value error, 密碼需要至少一個大寫字母
❌ 'NoNumbers' - Value error, 密碼需要至少一個數字
✅ 'Valid123Pass' - 通過


---

## 🏗️ 模型繼承

在真實 API 中，你會發現同一個資源（如 User）在不同情境需要不同欄位：

| 情境 | 需要的欄位 | 不需要的欄位 |
|------|-----------|-------------|
| **建立用戶** | name, email, password | id, created_at |
| **更新用戶** | name?, email? (都可選) | id, password |
| **回應用戶** | id, name, email, created_at | password |
| **資料庫** | 全部欄位 | - |

### 解決方案：模型繼承

與其寫四個獨立的模型（大量重複），不如用繼承：

```
UserBase (共用欄位)
├── UserCreate (建立時)
├── UserUpdate (更新時)
└── UserResponse (回應時)
    └── UserInDB (資料庫完整資料)
```

### 為什麼要這樣設計？

1. **DRY 原則** - 共用欄位只定義一次
2. **安全性** - 回應模型不包含密碼，不會誤洩
3. **靈活性** - 每個情境有專屬的驗證規則
4. **文檔清晰** - API 文檔準確反映每個端點的輸入輸出

In [9]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

# 基礎模型
class UserBase(BaseModel):
    """用戶基礎欄位"""
    name: str
    email: str

# 建立時使用（不需要 ID）
class UserCreate(UserBase):
    """建立用戶時的模型"""
    password: str

# 更新時使用（所有欄位都可選）
class UserUpdate(BaseModel):
    """更新用戶時的模型"""
    name: Optional[str] = None
    email: Optional[str] = None

# 回應時使用（包含 ID，不包含密碼）
class UserResponse(UserBase):
    """回應用的模型"""
    id: int
    created_at: datetime

# 資料庫模型（包含所有欄位）
class UserInDB(UserResponse):
    """資料庫完整模型"""
    hashed_password: str

print("📋 模型繼承結構：\n")
print("UserBase        → name, email")
print("├─ UserCreate   → name, email, password")
print("└─ UserResponse → name, email, id, created_at")
print("   └─ UserInDB  → ... + hashed_password")

📋 模型繼承結構：

UserBase        → name, email
├─ UserCreate   → name, email, password
└─ UserResponse → name, email, id, created_at
   └─ UserInDB  → ... + hashed_password


In [10]:
# 實際使用

# 1. 建立用戶（接收 UserCreate）
create_data = UserCreate(
    name="小明",
    email="ming@example.com",
    password="secret123"
)
print(f"建立請求: {create_data.model_dump()}")

# 2. 模擬儲存到資料庫
db_user = UserInDB(
    id=1,
    name=create_data.name,
    email=create_data.email,
    hashed_password="hashed_" + create_data.password,
    created_at=datetime.now()
)
print(f"\n資料庫資料: {db_user.model_dump()}")

# 3. 回傳給前端（使用 UserResponse，不包含密碼）
response = UserResponse(
    id=db_user.id,
    name=db_user.name,
    email=db_user.email,
    created_at=db_user.created_at
)
print(f"\n回應資料: {response.model_dump()}")

建立請求: {'name': '小明', 'email': 'ming@example.com', 'password': 'secret123'}

資料庫資料: {'name': '小明', 'email': 'ming@example.com', 'id': 1, 'created_at': datetime.datetime(2025, 12, 30, 17, 23, 31, 690413), 'hashed_password': 'hashed_secret123'}

回應資料: {'name': '小明', 'email': 'ming@example.com', 'id': 1, 'created_at': datetime.datetime(2025, 12, 30, 17, 23, 31, 690413)}


---

## 🪆 巢狀模型

模型可以包含其他模型作為欄位。

In [11]:
from pydantic import BaseModel
from typing import Optional

class Address(BaseModel):
    """地址模型"""
    city: str
    street: str
    zip_code: str

class Company(BaseModel):
    """公司模型"""
    name: str
    address: Address  # 巢狀模型

class User(BaseModel):
    """用戶模型（包含巢狀結構）"""
    name: str
    email: str
    address: Optional[Address] = None  # 可選的巢狀模型
    company: Optional[Company] = None

# 建立巢狀結構
user = User(
    name="小明",
    email="ming@example.com",
    address={
        "city": "台北",
        "street": "信義路",
        "zip_code": "110"
    },
    company={
        "name": "科技公司",
        "address": {
            "city": "新北",
            "street": "中山路",
            "zip_code": "220"
        }
    }
)

print("🪆 巢狀模型：\n")
print(f"用戶: {user.name}")
print(f"住址: {user.address.city} {user.address.street}")
print(f"公司: {user.company.name}")
print(f"公司地址: {user.company.address.city}")

🪆 巢狀模型：

用戶: 小明
住址: 台北 信義路
公司: 科技公司
公司地址: 新北


In [12]:
# 列表中的巢狀模型

class OrderItem(BaseModel):
    """訂單項目"""
    product_name: str
    quantity: int
    price: float

class Order(BaseModel):
    """訂單模型"""
    id: int
    customer_name: str
    items: list[OrderItem]  # 項目列表
    
    @property
    def total(self) -> float:
        """計算總金額"""
        return sum(item.price * item.quantity for item in self.items)

# 建立訂單
order = Order(
    id=1,
    customer_name="小明",
    items=[
        {"product_name": "書籍", "quantity": 2, "price": 300},
        {"product_name": "筆記本", "quantity": 3, "price": 50},
    ]
)

print(f"📦 訂單 #{order.id}")
print(f"客戶: {order.customer_name}")
print(f"\n項目:")
for item in order.items:
    print(f"  - {item.product_name} x{item.quantity} = ${item.price * item.quantity}")
print(f"\n總計: ${order.total}")

📦 訂單 #1
客戶: 小明

項目:
  - 書籍 x2 = $600.0
  - 筆記本 x3 = $150.0

總計: $750.0


---

## 🚀 在 FastAPI 中使用

Pydantic 模型可以直接用於 FastAPI 的請求和回應。

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

app = FastAPI()

# 定義模型
class ItemCreate(BaseModel):
    name: str = Field(min_length=1, max_length=100)
    price: float = Field(gt=0)
    description: Optional[str] = None

class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    description: Optional[str] = None
    created_at: datetime

# 模擬資料庫
items_db: dict[int, ItemResponse] = {}
current_id = 0

@app.post("/items", response_model=ItemResponse)
def create_item(item: ItemCreate):
    """建立項目
    
    FastAPI 會自動：
    1. 從請求體讀取 JSON
    2. 驗證資料格式
    3. 轉換為 ItemCreate 物件
    """
    global current_id
    current_id += 1
    
    new_item = ItemResponse(
        id=current_id,
        name=item.name,
        price=item.price,
        description=item.description,
        created_at=datetime.now()
    )
    items_db[current_id] = new_item
    return new_item

@app.get("/items/{item_id}", response_model=ItemResponse)
def get_item(item_id: int):
    """取得項目"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="項目不存在")
    return items_db[item_id]

In [ ]:
# 測試 API
from fastapi.testclient import TestClient

client = TestClient(app)

print("🧪 測試 API\n")

# 建立項目
response = client.post("/items", json={
    "name": "Python 書籍",
    "price": 599.0,
    "description": "FastAPI 教學"
})
print(f"POST /items: {response.json()}")

# 驗證失敗測試
response = client.post("/items", json={
    "name": "",  # 空字串
    "price": -100  # 負數
})
print(f"\n驗證失敗: 狀態碼 {response.status_code}")
for error in response.json()["detail"]:
    print(f"  - {error['loc'][-1]}: {error['msg']}")

---

## 📋 Pydantic 速查表

### 模型方法

| 方法 | 說明 |
|------|------|
| `model_dump()` | 轉換為字典 |
| `model_dump_json()` | 轉換為 JSON 字串 |
| `model_validate(dict)` | 從字典驗證並建立 |
| `model_validate_json(str)` | 從 JSON 驗證並建立 |
| `model_copy(update={})` | 複製並更新欄位 |

### Field 參數

| 參數 | 說明 |
|------|------|
| `default` | 預設值 |
| `min_length`, `max_length` | 字串長度限制 |
| `ge`, `gt`, `le`, `lt` | 數值範圍 |
| `pattern` | 正則表達式 |
| `description` | 欄位描述 |

---

## 🎯 練習題

建立一個 `Book` 模型，包含以下驗證：

In [ ]:
# 練習：建立 Book 模型

# 需求：
# - title: 字串，1-200 字元
# - author: 字串，必填
# - price: 浮點數，大於 0
# - pages: 整數，大於 0，預設 1
# - isbn: 字串，可選，格式為 13 位數字

# TODO: 在這裡寫你的程式碼
# class Book(BaseModel):
#     pass

<details>
<summary>📝 點擊查看解答</summary>

```python
from pydantic import BaseModel, Field
from typing import Optional

class Book(BaseModel):
    title: str = Field(min_length=1, max_length=200)
    author: str
    price: float = Field(gt=0)
    pages: int = Field(default=1, gt=0)
    isbn: Optional[str] = Field(default=None, pattern=r'^\d{13}$')

# 測試
book = Book(
    title="Python 入門",
    author="小明",
    price=399.0,
    pages=300,
    isbn="9781234567890"
)
print(book)
```

</details>

---

## 📋 本節重點

| 概念 | 說明 |
|------|------|
| BaseModel | Pydantic 模型基類 |
| Field | 欄位驗證規則 |
| @field_validator | 自訂驗證邏輯 |
| 模型繼承 | 重用模型定義 |
| 巢狀模型 | 模型包含模型 |

---

## ⏭️ 下一步

👉 **[2.3 依賴注入](03_dependency_injection.ipynb)** - 學習 FastAPI 的依賴注入系統